### Additional Baseline only for test: Combined

- Baseline: Combine
- We contatenate all high-rated samples from three labeling models

In [ ]:
import torch
from collections import defaultdict
import random
import numpy as np
import os
seed = 3
random.seed(3)
np.random.seed(3)

#llama labels
model_full_name ='meta-llama/Meta-Llama-3.1-8B-Instruct/'
new_train_dir = f"./model_finetune/new_train_data/{model_full_name}"
output_file = os.path.join(new_train_dir, 'all_train/output_labels_revised.pt')
llama_labels= torch.load(output_file)


llama_labels_dict = defaultdict(list)
for index, label in enumerate(llama_labels):
    llama_labels_dict[label].append(index)


##GPT 4o-mini labels
model_full_name='gpt-4o-mini'
new_train_dir = f"./model_finetune/new_train_data/{model_full_name}"
output_file = os.path.join(new_train_dir, 'all_train/output_labels_revised.pt')
gpt_labels= torch.load(output_file)

gpt_labels_dict = defaultdict(list)
for index, label in enumerate(gpt_labels):
    gpt_labels_dict[label].append(index)


#mistral labels
model_full_name='mistralai/Mistral-7B-Instruct-v0.3'
new_train_dir = f"./model_finetune/new_train_data/{model_full_name}"
output_file = os.path.join(new_train_dir, 'all_train/output_labels_revised.pt')
mistral_labels= torch.load(output_file)


mistral_labels_dict = defaultdict(list)
for index, label in enumerate(mistral_labels):
    mistral_labels_dict[label].append(index)



intersection_all_dict = {}

for label in [5,4,3,2,1]:
    intersection_gpt_llama = set(gpt_labels_dict[label]) & set(llama_labels_dict[label])
    intersection_gpt_mistral = set(gpt_labels_dict[label]) & set(mistral_labels_dict[label])
    intersection_llama_mistral = set(llama_labels_dict[label]) & set(mistral_labels_dict[label])
    intersection_all_dict[label] = intersection_gpt_llama & intersection_gpt_mistral & intersection_llama_mistral
    print(f"intersection_all: {len(intersection_all_dict[label])} --- intersection_gpt_llama size: {len(intersection_gpt_llama)} -- intersection_gpt_mistral size: {len(intersection_gpt_mistral)} -- intersection_llama_mistral size:{len(intersection_llama_mistral)}")


dataset_size = 10000
combined_label_indices = set(llama_labels_dict[5] + mistral_labels_dict[5] + gpt_labels_dict[5])

additional_label_indices = random.sample(intersection_all_dict[4], dataset_size-len(combined_label_indices))


ideal_label_indices = list(combined_label_indices) + additional_label_indices




## create combined datasets

In [ ]:
from datasets import load_dataset
import random
import numpy as np

seed =3
random.seed(seed)
np.random.seed(seed)

#### 
dataset_size = 10000
dataset_name ='all_train'
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"


json_dir = './data/train_data/'

all_train_dataset = load_dataset('json', data_files=json_dir+'all_train_data.jsonl')['train']


random_indices = np.random.permutation(len(all_train_dataset))[:dataset_size]

random_dataset = all_train_dataset.select(ideal_label_indices)

root_path = f"./model_finetune/new_train_data/{model_name}/{dataset_name}/"

random_dataset.to_json(root_path + f"combined_dataset.json")
